In [ ]:
from tensorflow import keras
# load
reconstructed_model = keras.models.load_model("./cheakpoint/XXX.h5") # 불러올 모델

In [ ]:
# model predict
y_pred = reconstructed_model.predict(x_test)

In [ ]:
new_y= []
for val in y_test:
    max = 0
    cnt = 0
    for idx, num in enumerate(val):
        if max < num:
            max = num
            cnt = idx + 1
    new_y.append(cnt)

In [ ]:
new_y_pred = []
for val in y_pred:
    max = 0
    cnt = 0
    for idx, num in enumerate(val):
        if max < num:
            max = num
            cnt = idx + 1
    new_y_pred.append(cnt)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
import seaborn as sns

cm2 = confusion_matrix(new_y, new_y_pred)
sns.heatmap(cm2, annot = True, fmt = 'd', cmap= 'Reds')
plt.xlabel('real')
plt.ylabel('predict')
plt.xticks([0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5], ['1', '2', '3', '4', '5', '6', '7', '8'])
plt.yticks([0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5], ['1', '2', '3', '4', '5', '6', '7', '8'])
plt.show()